In [ ]:
%load_ext lab_black
import os
import re
import json
import csv
import datetime
import requests
from time import time
from tqdm import tqdm

from common import save_graph, load_graph, get_from_playlists_from_time_range, id_to_url
from scrape_helpers import get_exported_youtube_playlist_ids, scrape_from_list

seconds_in_month = 60 * 60 * 24 * 30.4
playlists_path = "../data/Takeout/YouTube and YouTube Music/playlists"

In [ ]:
G = load_graph()

In [ ]:
def scrape_playlist(playlist_name, G):
    ids_to_add, times_added = get_exported_youtube_playlist_ids(
        f"{playlists_path}/{playlist_name}.csv"
    )

    scrape_from_list(ids_to_add, G, skip_if_fresher_than=seconds_in_month)

    # add data about the time they were added
    for id_, time_added in zip(ids_to_add, times_added):
        if id_ in G:
            G.nodes[id_]["time_added"] = time_added
            G.nodes[id_]["from"] = playlist_name

# Scrape all playlists

In [ ]:
for filename in os.listdir(playlists_path):
    playlist_name = re.match("(.*)\.csv", filename)[1]

    print()
    print("scraping: ", playlist_name)
    scrape_playlist(playlist_name, G)

In [ ]:
save_graph(G)

# Scrape neighborhood of recently liked videos


In [ ]:
seconds_in_year = 60 * 60 * 24 * 365
start_time = time() - seconds_in_year * 3
sources = get_from_playlists_from_time_range(G, start_time)
print(len(sources))

In [ ]:
_, out_nodes = zip(*G.out_edges(sources))
print(len(out_nodes))
out_nodes = set(out_nodes)
print(len(out_nodes))

In [ ]:
scrape_from_list(out_nodes, G, skip_if_fresher_than=seconds_in_month)

In [ ]:
save_graph(G)